In [1]:
import re
import xlwt
import chardet
from urllib import request
import random
import urllib

In [2]:
def getHtml(url):
    USER_AGENTS=[ "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; \
        .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1;\
         .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko)\
         Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) \
        Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732;\
         .NET4.0C; .NET4.0E; LBBROWSER)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; \
        SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; \
        Media Center PC 6.0; .NET4.0C; .NET4.0E)"] #浏览器选择
    proxies=[
            {"HTTP":"5.252.161.48:8080"}
            ] #代理IP
    req= request.Request(url)   #设置url地址
    req.add_header("User-Agent",random.choice(USER_AGENTS)) #随机选择一个浏览器
    proxy_support = request.ProxyHandler(random.choice(proxies))#随机选择一个IP
    opener= request.build_opener(proxy_support) #获取网站访问的对象
    request.install_opener(opener)
    res= request.urlopen(req) #处理浏览器返回的对象
    html=res.read()#读取网页内容
    return html #返回html

In [3]:
def get_Datalist(page_number,jobname):
    URL="https://search.51job.com/list/130800,000000,0000,00,9,99," \
           +urllib.parse.quote(jobname)+",2," + str(page_number) + ".html?lang=c&stype=&postchannel\
           =0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=\
           99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=\
           9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare="
    html=getHtml(URL)  #传入需要分析的网页
    code=chardet.detect(html)["encoding"] #获取网页编码
    html = html.decode(code,'replace').encode("utf-8") #解编码，转码为utf-8编码
    #设置正则表达式
    reg= re.compile(r'<p class="t1 ">.*?<a target="_blank" title="(.*?)" .*?'
                     r'<span class="t2"><a target="_blank" title="(.*?)" .*?'  
                     r'<span class="t3">(.*?)</span>.*?'
                     r'<span class="t4">(.*?)</span>.*?'
                     r'<span class="t5">(.*?)</span>', re.S)
    result = re.findall(reg,html.decode("utf-8",'replace'))#替换非法字符
    return result



In [4]:
datalist=[] #全局数据列表
def solve_data(page_number,jobname):
    #向全局变量添加数据
    global datalist
    for k in range(int (page_number)):
        data=get_Datalist(k+1,jobname)
        for i in data:
            datalist.append(i)

In [5]:
def save_Excel(jobname,filename):   #设置存储函数
    book =xlwt.Workbook(encoding = "utf-8") # 创建工作本
    sheet = book.add_sheet("51job"+str(jobname)+"职位信息")
    col=('职位名','公司名','工作地点','薪资','发布时间')
    for i in range(len(col)):
        sheet.write(0,i,col[i])
        
    for i in range(len(datalist)):
        for j in range(len(datalist[i])):
            sheet.write(i+1,j,datalist[i][j])
    book.save(u'51job'+filename+u'职位信息.xls')

In [6]:
def main(jobname,page_number,filename):
    solve_data(page_number,jobname)
    save_Excel(jobname,filename)

In [7]:
main(u'机器学习工程师',"2",u'机器学习工程师1') #爬取

HTTPError: HTTP Error 400: Bad Request